In [92]:
import pandas as pd
import warnings
from src.utils import *
import openai
import src.gpt_siemens as siemens
warnings.filterwarnings('ignore')

## 1) Data

In [108]:
df=pd.read_pickle('data/df_Mahle_full.pkl')

In [109]:
df.shape

(931744, 19)

In [110]:
df.head(3)

,Product Id,Manufacturer Name,Manufacturer PID,model,Short Description,Long Description,Additional description,score,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,matched_Product Id,matched_model,matched_Manufacturer PID,matched_Supplier PID,matched_Manufacturer Name,matched_Supplier Name,pair
0,61085983,CARR-LANE,CL-40-BP-1,CL-40-BP-1,"PLUNGER, 1/4 X 20 PN CL-40-BP-1",NaN,NaN,85.71,cl40bp1,cl70bp1,carrlane,carrlane,61085352,CL70BP1,CL70BP1,NaN,CARR LANE,NaN,"(61085352, 61085983)"
1,61085983,CARR-LANE,CL-40-BP-1,CL-40-BP-1,"PLUNGER, 1/4 X 20 PN CL-40-BP-1",NaN,NaN,71.43,cl40bp1,cl40sbp2,carrlane,carrlane,61085952,CL-40-SBP-2,CL-40-SBP-2,NaN,CARR LANE,NaN,"(61085952, 61085983)"
2,61085983,CARR-LANE,CL-40-BP-1,CL-40-BP-1,"PLUNGER, 1/4 X 20 PN CL-40-BP-1",NaN,NaN,58.93,cl40bp1,clm10bp1,carrlane,carrline,61069715,NaN,CLM-10-BP1,NaN,CARR LINE,NaN,"(61069715, 61085983)"


In [111]:
df['Product Id'].value_counts()

Product Id
61317863     848
61317942     844
61317856     844
61317860     844
61317858     843
            ... 
101016978      1
61096883       1
61096918       1
61082645       1
60992313       1
Name: count, Length: 75135, dtype: int64

In [112]:
df[df['Product Id']==61085352]

,Product Id,Manufacturer Name,Manufacturer PID,model,Short Description,Long Description,Additional description,score,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,matched_Product Id,matched_model,matched_Manufacturer PID,matched_Supplier PID,matched_Manufacturer Name,matched_Supplier Name,pair
701394,61085352,CARR LANE,CL70BP1,CL70BP1,"PLUNGER, CARR LANE CL-70-BP-1 BALL NOSE",NaN,NaN,71.43,cl70bp1,cl70sbpn1,carrlane,carrlane,61085237,CL-70-SBPN-1,CL-70-SBPN-1,NaN,CARR LANE,NaN,"(61085237, 61085352)"
701395,61085352,CARR LANE,CL70BP1,CL70BP1,"PLUNGER, CARR LANE CL-70-BP-1 BALL NOSE",NaN,NaN,85.71,cl70bp1,cl40bp1,carrlane,carrlane,61085983,CL-40-BP-1,CL-40-BP-1,NaN,CARR-LANE,NaN,"(61085352, 61085983)"
701396,61085352,CARR LANE,CL70BP1,CL70BP1,"PLUNGER, CARR LANE CL-70-BP-1 BALL NOSE",NaN,NaN,85.71,cl70bp1,cl70bp2,carrlane,carrlane,61085351,CL-70-BP-2,CL-70-BP-2,NaN,CARR LANE,NaN,"(61085351, 61085352)"


### 2) Eliminate all duplicates

In [7]:
df['pair'] = df.apply(lambda row: tuple(sorted([row['Product Id'], row['matched_Product Id']])), axis=1)
df_cleaned = df.drop_duplicates(subset='pair', keep='first')  # Drop duplicates based on the pair column, keeping the first occurrence
duplicate_pairs = df[df.duplicated(subset='pair', keep=False)]
# df_cleaned = df_cleaned.drop(columns=['pair'])

Checking pairs

In [103]:
df[df['pair']==(61023556, 61068240)]

,Product Id,Manufacturer Name,Manufacturer PID,model,Short Description,Long Description,Additional description,score,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,matched_Product Id,matched_model,matched_Manufacturer PID,matched_Supplier PID,matched_Manufacturer Name,matched_Supplier Name,pair
206,61068240,ROCKWELL AUTOMATION,NaN,140M-C2E-A63,140M Motorschutzschalter,NaN,NaN,90.0,140mc2ea63,140mc2eb63,rockwellautomation,rockwellautomation,61023556,140M-C2E-B63,NaN,NaN,ROCKWELL AUTOMATION,NaN,"(61023556, 61068240)"
208,61068240,ROCKWELL AUTOMATION,NaN,140M-C2E-A63,140M Motorschutzschalter,NaN,NaN,90.0,140mc2ea63,140mc2eb63,rockwellautomation,rockwellautomation,61023556,140M-C2E-B63,NaN,NaN,ROCKWELL AUTOMATION,NaN,"(61023556, 61068240)"
419174,61023556,ROCKWELL AUTOMATION,NaN,140M-C2E-B63,140M Motorschutzschalter,NaN,NaN,90.0,140mc2eb63,140mc2ea63,rockwellautomation,rockwellautomation,61068240,140M-C2E-A63,NaN,NaN,ROCKWELL AUTOMATION,NaN,"(61023556, 61068240)"


In [9]:
df.pair.value_counts()

pair
(101022224, 101022319)    80
(101019798, 101020108)    56
(101021333, 101021334)    48
(101021335, 101021339)    48
(101024412, 101024413)    48
                          ..
(61063286, 61380709)       1
(60985235, 61063286)       1
(61063286, 77134079)       1
(61049793, 61064397)       1
(61092588, 61094077)       1
Name: count, Length: 240944, dtype: int64

In [169]:
df[df['pair']==(101022224, 101022319)]

,Product Id,Manufacturer Name,Manufacturer PID,model,Short Description,Long Description,Additional description,score,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,matched_Product Id,matched_model,matched_Manufacturer PID,matched_Supplier PID,matched_Manufacturer Name,matched_Supplier Name,pair
476576,101022319,FESTO,156622,ADVU-32-40-A-P-A,Kompaktzylinder,NaN,ADVU-32-40-A-P-A; SEAL KIT,100.0,156622,156622,festo,festo,101022224,ADVU-32-40-A-P-A,156622,NaN,FESTO,Festo Corporation,"(101022224, 101022319)"
476577,101022319,FESTO,156622,ADVU-32-40-A-P-A,Kompaktzylinder,NaN,ADVU-32-40-A-P-A; SEAL KIT,100.0,156622,156622,festo,festo,101022224,ADVU-32-40-A-P-A,156622,NaN,FESTO,Festo Corporation,"(101022224, 101022319)"
476578,101022319,FESTO,156622,ADVU-32-40-A-P-A,Kompaktzylinder,NaN,ADVU-32-40-A-P-A; SEAL KIT,100.0,156622,156622,festo,festo,101022224,ADVU-32-40-A-P-A,156622,NaN,FESTO,Festo Corporation,"(101022224, 101022319)"
476579,101022319,FESTO,156622,ADVU-32-40-A-P-A,Kompaktzylinder,NaN,ADVU-32-40-A-P-A; SEAL KIT,100.0,156622,156622,festo,festo,101022224,ADVU-32-40-A-P-A,156622,NaN,FESTO,Festo Corporation,"(101022224, 101022319)"
476586,101022319,FESTO,156622,ADVU-32-40-A-P-A,Kompaktzylinder,NaN,ADVU-32-40-A-P-A; SEAL KIT,100.0,156622,156622,festo,festo,101022224,ADVU-32-40-A-P-A,156622,NaN,FESTO,Festo Corporation,"(101022224, 101022319)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
818646,101022224,FESTO,156622,ADVU-32-40-A-P-A,Kompaktzylinder,NaN,ADVU-32-40-A-P-A;,100.0,advu3240apa,advu3240apa,festo,festo,101022319,ADVU-32-40-A-P-A,156622,NaN,FESTO,Festo Corporation,"(101022224, 101022319)"
818653,101022224,FESTO,156622,ADVU-32-40-A-P-A,Kompaktzylinder,NaN,ADVU-32-40-A-P-A;,100.0,advu3240apa,advu3240apa,festo,festo,101022319,ADVU-32-40-A-P-A,156622,NaN,FESTO,Festo Corporation,"(101022224, 101022319)"
818654,101022224,FESTO,156622,ADVU-32-40-A-P-A,Kompaktzylinder,NaN,ADVU-32-40-A-P-A;,100.0,advu3240apa,advu3240apa,festo,festo,101022319,ADVU-32-40-A-P-A,156622,NaN,FESTO,Festo Corporation,"(101022224, 101022319)"
818655,101022224,FESTO,156622,ADVU-32-40-A-P-A,Kompaktzylinder,NaN,ADVU-32-40-A-P-A;,100.0,advu3240apa,advu3240apa,festo,festo,101022319,ADVU-32-40-A-P-A,156622,NaN,FESTO,Festo Corporation,"(101022224, 101022319)"


In [12]:
df[df['pair']==(101022224, 101022319)].iloc[75]

Product Id                                101022224
Manufacturer Name                             FESTO
Manufacturer PID                             156622
model                              ADVU-32-40-A-P-A
Short Description                   Kompaktzylinder
Long Description                                NaN
Additional description           ADVU-32-40-A-P-A; 
score                                         100.0
matched_pid_left                        advu3240apa
matched_pid_right                       advu3240apa
matched_brand_left                            festo
matched_brand_right                           festo
matched_Product Id                        101022319
matched_model                      ADVU-32-40-A-P-A
matched_Manufacturer PID                     156622
matched_Supplier PID                            NaN
matched_Manufacturer Name                     FESTO
matched_Supplier Name             Festo Corporation
pair                         (101022224, 101022319)
Name: 818646

In [13]:
df_cleaned.shape

(240944, 19)

In [14]:
df_cleaned.pair.value_counts()

pair
(61085352, 61085983)     1
(61003168, 101018239)    1
(61002266, 77139837)     1
(60687193, 61002266)     1
(61002266, 61008049)     1
                        ..
(61088953, 61088987)     1
(61088953, 61089018)     1
(61088958, 61088960)     1
(61086485, 61088962)     1
(61092588, 61094077)     1
Name: count, Length: 240944, dtype: int64

In [100]:
df_cleaned[df_cleaned['pair']== (61068113, 61068114)]

,Product Id,Manufacturer Name,Manufacturer PID,model,Short Description,Long Description,Additional description,score,matched_pid_left,matched_pid_right,...,matched_Supplier PID,matched_Manufacturer Name,matched_Supplier Name,pair,cleaned_Manufacturer Name,cleaned_Manufacturer PID,cleaned_model,match_Manufacturer Name_cleaned,match_Manufacturer PID_cleaned,match_model_cleaned
18,61068113,CORE,CDM08MH-MG-D2XX-01-01,CDM08MH-MG-D2XX-01-01,MASTER ROJO HEATER CORE D2XX,NaN,NaN,100.0,cdm08mhmgd2xx0101,cdm08mhmgd2xx0101,...,NaN,CORE,NaN,"(61068113, 61068114)",core,cdm08mhmgd2xx0101,cdm08mhmgd2xx0101,core,cdm08mhmgd2xx0101,cdm08mhmgd2xx0101


#### Double check pairs

Extract pairs and compare every time the pid and brand inside every pair to find out if there is any mismatch

In [113]:
df_test=df.copy()

In [117]:
df_test=df.iloc[0:10000]

In [118]:
df_test.shape

(10000, 19)

In [143]:
# df_test['pair'] = df_test.apply(lambda row: tuple(sorted([row['Product Id'], row['matched_Product Id']])), axis=1)
# score_diff = df_test[df_test.duplicated(subset=['pair'], keep=False)]
# score_diff = score_diff.groupby('pair').filter(lambda x: len(x['score'].unique()) > 1)
# score_diff = score_diff.drop_duplicates(subset=['pair', 'score'], keep='first')

In [151]:
df['pair'] = df.apply(lambda row: tuple(sorted([row['Product Id'], row['matched_Product Id']])), axis=1)
score_diff = df[df.duplicated(subset=['pair'], keep=False)]
score_diff_tot = score_diff.groupby('pair').filter(lambda x: len(x['score'].unique()) > 1)
score_diff_uniq = score_diff_tot.drop_duplicates(subset=['pair', 'score'], keep='first')

In [168]:
score_diff_uniq.head()

,Product Id,Manufacturer Name,Manufacturer PID,model,Short Description,Long Description,Additional description,score,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,matched_Product Id,matched_model,matched_Manufacturer PID,matched_Supplier PID,matched_Manufacturer Name,matched_Supplier Name,pair
1,61085983,CARR-LANE,CL-40-BP-1,CL-40-BP-1,"PLUNGER, 1/4 X 20 PN CL-40-BP-1",NaN,NaN,71.43,cl40bp1,cl40sbp2,carrlane,carrlane,61085952,CL-40-SBP-2,CL-40-SBP-2,NaN,CARR LANE,NaN,"(61085952, 61085983)"
2,61085983,CARR-LANE,CL-40-BP-1,CL-40-BP-1,"PLUNGER, 1/4 X 20 PN CL-40-BP-1",NaN,NaN,58.93,cl40bp1,clm10bp1,carrlane,carrline,61069715,NaN,CLM-10-BP1,NaN,CARR LINE,NaN,"(61069715, 61085983)"
6,61085951,Banner,Q23SP6RQ,Q23SP6RQ,"BANNER, Q23SP6RQ",NaN,NaN,75.00,q23sp6rq,q23sp6d,banner,banner,61098160,Q23SP6D,Q23SP6D,NaN,Banner,NaN,"(61085951, 61098160)"
13,61068180,Bosch,R165111320,NaN,BOSCH REXROTH LINEAR R165111320 SIZE 15,NaN,NaN,90.00,r165111320,165111320,bosch,bosch,60993267,NaN,1651-113-20,NaN,BOSCH REXROTH,NaN,"(60993267, 61068180)"
55,61068137,FESTO,163302,DNC-32-100-PPV-A-Q-52,Normzylinder,NaN,NaN,100.00,163302,163302,festo,festo,61003496,DNC-32-10-PPV-A-Q-S2,163302,NaN,FESTO,NaN,"(61003496, 61068137)"


In [165]:
score_aggregated = score_diff_uniq.groupby('pair')['score'].apply(lambda x: list(x.unique())).reset_index()
score_aggregated = pd.DataFrame(score_aggregated['score'].to_list(), index=score_aggregated['pair']).reset_index()
score_aggregated.columns = ['pair'] + [f'score_{i+1}' for i in range(score_aggregated.shape[1] - 1)]
score_aggregated

,pair,score_1,score_2,score_3,score_4,score_5,score_6,score_7
0,"(60685766, 61059641)",71.43,75.00,NaN,NaN,NaN,NaN,NaN
1,"(60685769, 61023604)",75.00,73.33,NaN,NaN,NaN,NaN,NaN
2,"(60685780, 60984325)",100.00,77.78,NaN,NaN,NaN,NaN,NaN
3,"(60685783, 101026298)",78.57,76.92,NaN,NaN,NaN,NaN,NaN
4,"(60685785, 61070128)",84.62,83.33,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
26320,"(116890095, 116890096)",96.00,90.00,NaN,NaN,NaN,NaN,NaN
26321,"(116890109, 116890123)",76.92,78.57,NaN,NaN,NaN,NaN,NaN
26322,"(116890114, 116890126)",83.33,94.12,NaN,NaN,NaN,NaN,NaN
26323,"(116950721, 116950722)",85.71,87.80,NaN,NaN,NaN,NaN,NaN


In [166]:
no_nan_rows = score_aggregated.dropna()
no_nan_rows

,pair,score_1,score_2,score_3,score_4,score_5,score_6,score_7
4832,"(60979151, 101023204)",70.0,100.0,66.0,96.0,57.14,52.98,95.83


In [ ]:
(60979151, 101023204)	

In [152]:
score_diff_tot.shape

(111153, 19)

In [153]:
score_diff_uniq.shape

(54784, 19)

In [167]:
score_diff_uniq[score_diff_uniq['pair']==(60979151, 101023204)]

,Product Id,Manufacturer Name,Manufacturer PID,model,Short Description,Long Description,Additional description,score,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,matched_Product Id,matched_model,matched_Manufacturer PID,matched_Supplier PID,matched_Manufacturer Name,matched_Supplier Name,pair
128911,101023204,BALLUFF,BCC02MM,BCC M323-0000-10-001-PX0334-050,Anschlussleitungen,NaN,BKS-S49-1-PU-05; replaced by #BCC M323-0000-10...,70.00,bcc02mm,bcc02mm,balluffinc,balluff,60979151,BCC M323-0000-10-001-PX0334-05,BCC02MM,NaN,BALLUFF,NaN,"(60979151, 101023204)"
128912,101023204,BALLUFF,BCC02MM,BCC M323-0000-10-001-PX0334-050,Anschlussleitungen,NaN,BKS-S49-1-PU-05; replaced by #BCC M323-0000-10...,100.00,bcc02mm,bcc02mm,balluff,balluff,60979151,BCC M323-0000-10-001-PX0334-05,BCC02MM,NaN,BALLUFF,NaN,"(60979151, 101023204)"
128937,101023204,BALLUFF,BCC02MM,BCC M323-0000-10-001-PX0334-050,Anschlussleitungen,NaN,BKS-S49-1-PU-05; replaced by #BCC M323-0000-10...,66.00,bccm323000010001px0334050,bccm323000010001px033405,balluffinc,balluff,60979151,BCC M323-0000-10-001-PX0334-05,BCC02MM,NaN,BALLUFF,NaN,"(60979151, 101023204)"
128938,101023204,BALLUFF,BCC02MM,BCC M323-0000-10-001-PX0334-050,Anschlussleitungen,NaN,BKS-S49-1-PU-05; replaced by #BCC M323-0000-10...,96.00,bccm323000010001px0334050,bccm323000010001px033405,balluff,balluff,60979151,BCC M323-0000-10-001-PX0334-05,BCC02MM,NaN,BALLUFF,NaN,"(60979151, 101023204)"
438452,60979151,BALLUFF,BCC02MM,BCC M323-0000-10-001-PX0334-05,STECKVERBINDER BALLUFF BKS-B 49-1-PU-05,NaN,BCC M323-0000-10-001-PX0334-05,57.14,bcc02mm,bcc02mm,balluff,balluffinc,101023204,BCC M323-0000-10-001-PX0334-050,BCC02MM,NaN,BALLUFF,"BALLUFF, INC. **","(60979151, 101023204)"
438466,60979151,BALLUFF,BCC02MM,BCC M323-0000-10-001-PX0334-05,STECKVERBINDER BALLUFF BKS-B 49-1-PU-05,NaN,BCC M323-0000-10-001-PX0334-05,52.98,bccm323000010001px033405,bccm323000010001px0334050,balluff,balluffinc,101023204,BCC M323-0000-10-001-PX0334-050,BCC02MM,NaN,BALLUFF,"BALLUFF, INC. **","(60979151, 101023204)"
438467,60979151,BALLUFF,BCC02MM,BCC M323-0000-10-001-PX0334-05,STECKVERBINDER BALLUFF BKS-B 49-1-PU-05,NaN,BCC M323-0000-10-001-PX0334-05,95.83,bccm323000010001px033405,bccm323000010001px0334050,balluff,balluff,101023204,BCC M323-0000-10-001-PX0334-050,BCC02MM,NaN,BALLUFF,"BALLUFF, INC. **","(60979151, 101023204)"


In [154]:
score_diff_uniq[score_diff_uniq['pair']==(61085952, 61085983)]

,Product Id,Manufacturer Name,Manufacturer PID,model,Short Description,Long Description,Additional description,score,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,matched_Product Id,matched_model,matched_Manufacturer PID,matched_Supplier PID,matched_Manufacturer Name,matched_Supplier Name,pair
1,61085983,CARR-LANE,CL-40-BP-1,CL-40-BP-1,"PLUNGER, 1/4 X 20 PN CL-40-BP-1",NaN,NaN,71.43,cl40bp1,cl40sbp2,carrlane,carrlane,61085952,CL-40-SBP-2,CL-40-SBP-2,NaN,CARR LANE,NaN,"(61085952, 61085983)"
584546,61085952,CARR LANE,CL-40-SBP-2,CL-40-SBP-2,"BALL PLUNGER, CARR-LANE # CL-40-SBP-2",NaN,NaN,75.00,cl40sbp2,cl40bp1,carrlane,carrlane,61085983,CL-40-BP-1,CL-40-BP-1,NaN,CARR-LANE,NaN,"(61085952, 61085983)"


In [156]:
score_diff_uniq[score_diff_uniq['pair']==(61085951, 61098160)]

,Product Id,Manufacturer Name,Manufacturer PID,model,Short Description,Long Description,Additional description,score,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,matched_Product Id,matched_model,matched_Manufacturer PID,matched_Supplier PID,matched_Manufacturer Name,matched_Supplier Name,pair
6,61085951,Banner,Q23SP6RQ,Q23SP6RQ,"BANNER, Q23SP6RQ",NaN,NaN,75.00,q23sp6rq,q23sp6d,banner,banner,61098160,Q23SP6D,Q23SP6D,NaN,Banner,NaN,"(61085951, 61098160)"
647696,61098160,Banner,Q23SP6D,Q23SP6D,SENSOR OPTICO 30VDC,NaN,NaN,71.43,q23sp6d,q23sp6rq,banner,banner,61085951,Q23SP6RQ,Q23SP6RQ,NaN,Banner,NaN,"(61085951, 61098160)"


In [16]:
# duplicate_pairs

In [17]:
# df_cleaned.to_pickle('data/df_clean_duplicates.pkl')

### 3) Analyze the scoring

#### 3.1) Divide by scoring values

In [18]:
df_100 = df_cleaned[df_cleaned['score'] == 100]
df_95_100 = df_cleaned[(df_cleaned['score'] > 95) & (df_cleaned['score'] < 100)]

df_90 = df_cleaned[(df_cleaned['score'] >= 90) & (df_cleaned['score'] < 91)]
df_91 = df_cleaned[(df_cleaned['score'] >= 91) & (df_cleaned['score'] < 92)]
df_92 = df_cleaned[(df_cleaned['score'] >= 92) & (df_cleaned['score'] < 93)]
df_93 = df_cleaned[(df_cleaned['score'] >= 93) & (df_cleaned['score'] < 94)]
df_94 = df_cleaned[(df_cleaned['score'] >= 94) & (df_cleaned['score'] < 95)]

In [19]:
df_100.shape

(48632, 19)

In [20]:
df_95_100.shape

(727, 19)

In [21]:
df_93.shape

(8229, 19)

### 3.2) Score=100

In [22]:
df_100[['Product Id', 'Manufacturer Name', 'Manufacturer PID', 'model',
       'score', 'matched_pid_left', 'matched_pid_right', 'matched_brand_left',
       'matched_brand_right', 'matched_Product Id', 'matched_model',
       'matched_Manufacturer PID', 'matched_Supplier PID',
       'matched_Manufacturer Name', 'matched_Supplier Name']]

,Product Id,Manufacturer Name,Manufacturer PID,model,score,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,matched_Product Id,matched_model,matched_Manufacturer PID,matched_Supplier PID,matched_Manufacturer Name,matched_Supplier Name
7,61068180,Bosch,R165111320,NaN,100.0,r165111320,r165111320,bosch,bosch,60687083,NaN,R165111320,NaN,BOSCH,NaN
8,61068180,Bosch,R165111320,NaN,100.0,r165111320,r165111320,bosch,bosch,61012375,NaN,R165111320,NaN,BOSCH REXROTH,NaN
18,61068113,CORE,CDM08MH-MG-D2XX-01-01,CDM08MH-MG-D2XX-01-01,100.0,cdm08mhmgd2xx0101,cdm08mhmgd2xx0101,core,core,61068114,CDM08MH-MG-D2XX-01-01,CDM08MH-MG-D2XX-01-01,NaN,CORE,NaN
20,61049716,ROCKWELL AUTOMATION,NaN,1762-IQ32T,100.0,1762iq32t,1762iq32t,rockwellautomation,rockwellautomation,61086972,1762-IQ32T,NaN,NaN,ROCKWELL AUTOMATION,NaN
41,61068137,FESTO,163302,DNC-32-100-PPV-A-Q-52,100.0,163302,163302,festo,festo,101026048,DNC-32-200-PPV-A-Q,163302,NaN,FESTO,FESTO PNEUMATIC SA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
928194,61066423,HUSSMANN,BJW 28x1.4x0.24,#2,100.0,bjw28x14x024,bjw28x14x024,hussmann,hussmann,61066424,#6,BJW 28x1.4x0.24,NaN,HUSSMANN,NaN
928355,61066599,HUSSMANN,CB 16.x1.30x0.24,#3,100.0,cb16x130x024,cb16x130x024,hussmann,hussmann,61066579,#1,CB 16.x1.30x0.24,NaN,HUSSMANN,NaN
929450,61025526,MAHLE QUOTATION,5 9120 000,AZ080789,100.0,az080789,az080789,mahlequotation,mahlequotation,61025517,AZ080789,AZ080789(knife-.35mm),NaN,MAHLE QUOTATION,NaN
929451,61025526,MAHLE QUOTATION,5 9120 000,AZ080789,100.0,az080789,az080789,mahlequotation,mahlequotation,61025509,AZ080789,5 9010 000,NaN,MAHLE QUOTATION,NaN


In [23]:
df_100.head(3)

,Product Id,Manufacturer Name,Manufacturer PID,model,Short Description,Long Description,Additional description,score,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,matched_Product Id,matched_model,matched_Manufacturer PID,matched_Supplier PID,matched_Manufacturer Name,matched_Supplier Name,pair
7,61068180,Bosch,R165111320,NaN,BOSCH REXROTH LINEAR R165111320 SIZE 15,NaN,NaN,100.0,r165111320,r165111320,bosch,bosch,60687083,NaN,R165111320,NaN,BOSCH,NaN,"(60687083, 61068180)"
8,61068180,Bosch,R165111320,NaN,BOSCH REXROTH LINEAR R165111320 SIZE 15,NaN,NaN,100.0,r165111320,r165111320,bosch,bosch,61012375,NaN,R165111320,NaN,BOSCH REXROTH,NaN,"(61012375, 61068180)"
18,61068113,CORE,CDM08MH-MG-D2XX-01-01,CDM08MH-MG-D2XX-01-01,MASTER ROJO HEATER CORE D2XX,NaN,NaN,100.0,cdm08mhmgd2xx0101,cdm08mhmgd2xx0101,core,core,61068114,CDM08MH-MG-D2XX-01-01,CDM08MH-MG-D2XX-01-01,NaN,CORE,NaN,"(61068113, 61068114)"


In [24]:
df_100.columns

Index(['Product Id', 'Manufacturer Name', 'Manufacturer PID', 'model',
       'Short Description', 'Long Description', 'Additional description',
       'score', 'matched_pid_left', 'matched_pid_right', 'matched_brand_left',
       'matched_brand_right', 'matched_Product Id', 'matched_model',
       'matched_Manufacturer PID', 'matched_Supplier PID',
       'matched_Manufacturer Name', 'matched_Supplier Name', 'pair'],
      dtype='object')

In [25]:
# Apply the cleaning function to your dataframe
df_100_cleaned = clean_columns(df_100)

In [26]:
df_100_cleaned = df_100_cleaned[['score', 'cleaned_Manufacturer Name', 'cleaned_Manufacturer PID',
                                  'cleaned_model', 'match_Manufacturer Name_cleaned',
                                  'match_Manufacturer PID_cleaned', 'match_model_cleaned']]

In [27]:
df_100_cleaned.columns = [col.replace(" ", "_") for col in df_100_cleaned.columns]

In [28]:
df_100_cleaned.head(2)

,score,cleaned_Manufacturer_Name,cleaned_Manufacturer_PID,cleaned_model,match_Manufacturer_Name_cleaned,match_Manufacturer_PID_cleaned,match_model_cleaned
7,100.0,bosch,r165111320,NaN,bosch,r165111320,NaN
8,100.0,bosch,r165111320,NaN,boschrexroth,r165111320,NaN


In [29]:
df_100_cleaned.columns

Index(['score', 'cleaned_Manufacturer_Name', 'cleaned_Manufacturer_PID',
       'cleaned_model', 'match_Manufacturer_Name_cleaned',
       'match_Manufacturer_PID_cleaned', 'match_model_cleaned'],
      dtype='object')

In [30]:
# import pandas as pd

# def calculate_new_score(row):
#     score = 100  # Start with a score of 100%
    
#     # Define the columns to compare
#     columns_to_compare = [
#         ('Manufacturer Name', 'matched_Manufacturer Name'),
#         ('model', 'matched_model')
#     ]
    
#     # Compare each pair of columns (case-insensitive and ignoring NaN)
#     for col1, col2 in columns_to_compare:
#         if pd.isna(row[col1]) or pd.isna(row[col2]):
#             continue
        
#         if row[col1].lower() != row[col2].lower():  # If the values don't match, reduce the score
#             score -= 33
    
#     # Standardize and compare Manufacturer PID (remove hyphens)
#     manufacturer_pid = str(row['Manufacturer PID']).replace('-', '')
#     matched_pid_left = str(row['matched_pid_left']).replace('-', '')
#     matched_pid_right = str(row['matched_pid_right']).replace('-', '')
#     matched_manufacturer_pid = str(row['matched_Manufacturer PID']).replace('-', '')
    
#     # If Manufacturer PID matches any of the other columns, no score reduction
#     if manufacturer_pid == matched_pid_left or manufacturer_pid == matched_pid_right or manufacturer_pid == matched_manufacturer_pid:
#         pass  # No change to score if there's a match
#     else:
#         score -= 33  # Reduce score if there is no match
    
#     return score


In [31]:
# Apply the function to create the new_score column
df_100_cleaned['new_score'] = df_100_cleaned.apply(calculate_new_score, axis=1)
df_100_cleaned.head(35)

,score,cleaned_Manufacturer_Name,cleaned_Manufacturer_PID,cleaned_model,match_Manufacturer_Name_cleaned,match_Manufacturer_PID_cleaned,match_model_cleaned,new_score
7,100.0,bosch,r165111320,NaN,bosch,r165111320,NaN,100
8,100.0,bosch,r165111320,NaN,boschrexroth,r165111320,NaN,67
18,100.0,core,cdm08mhmgd2xx0101,cdm08mhmgd2xx0101,core,cdm08mhmgd2xx0101,cdm08mhmgd2xx0101,100
20,100.0,rockwellautomation,NaN,1762iq32t,rockwellautomation,NaN,1762iq32t,100
41,100.0,festo,163302,dnc32100ppvaq52,festo,163302,dnc32200ppvaq,67
42,100.0,festo,163302,dnc32100ppvaq52,festo,163302,dnc32200ppvakp,67
43,100.0,festo,163302,dnc32100ppvaq52,festo,163302,dnc32600ppvar3,67
44,100.0,festo,163302,dnc32100ppvaq52,festo,163302,dnc32140ppvakp,67
45,100.0,festo,163302,dnc32100ppvaq52,festo,163302,dnc32200ppvakp,67
46,100.0,festo,163302,dnc32100ppvaq52,festo,163302,dnc32100pas6,67


In [32]:
df_100_cleaned.loc[184]  # Is new_core=100 because it skips all NaN values

score                                   100.0
cleaned_Manufacturer_Name                 smc
cleaned_Manufacturer_PID           vqc2a01n51
cleaned_model                             NaN
match_Manufacturer_Name_cleaned           smc
match_Manufacturer_PID_cleaned            NaN
match_model_cleaned                vqc2a01n51
new_score                                 100
Name: 184, dtype: object

In [33]:
df_100_cleaned.new_score.describe()

count    48632.000000
mean        87.097076
std         17.655489
min          1.000000
25%         67.000000
50%        100.000000
75%        100.000000
max        100.000000
Name: new_score, dtype: float64

In [34]:
df_100_cleaned.new_score.value_counts()

new_score
100    30778
67     16702
34      1143
1          9
Name: count, dtype: int64

In [35]:
df_100_cleaned[df_100_cleaned['new_score']==1]

,score,cleaned_Manufacturer_Name,cleaned_Manufacturer_PID,cleaned_model,match_Manufacturer_Name_cleaned,match_Manufacturer_PID_cleaned,match_model_cleaned,new_score
39559,100.0,prowadzeniebba8fuehrungsstange,5100000,bba85170000200,mahlegmbh,510000,al15300000800,1
156103,100.0,waobrotuladera,5100000,al993204010200,mahlegmbh,510000,al15300000800,1
157005,100.0,konierzporednizwischenflansch,5100000,bba85170000400,mahlegmbh,510000,al15300000800,1
159080,100.0,obudowadooyska,5100000,fb53203020100,mahlegmbh,510000,al15300000800,1
272909,100.0,continental,5100000,fd133521000100,mahlegmbh,510000,al15300000800,1
390456,100.0,konierzoyskalagerflansch,5100000,bba85170000300,mahlegmbh,510000,al15300000800,1
504002,100.0,konierzflanschbba8,5100000,bba851000100,mahlegmbh,510000,al15300000800,1
619439,100.0,mahlegmbh,510000,al15300000800,ruraladerapipelader2074mm,5100000,al994214000100,1
662043,100.0,endresshauser,ftl2600020,51097f02,emdresshauser,9422020020,ftl2600020,1


In [36]:
df_100_cleaned.loc[919252]

score                                     100.0
cleaned_Manufacturer_Name              hussmann
cleaned_Manufacturer_PID           cb16x130x024
cleaned_model                               105
match_Manufacturer_Name_cleaned        hussmann
match_Manufacturer_PID_cleaned     cb16x130x024
match_model_cleaned                          10
new_score                                    67
Name: 919252, dtype: object

In [37]:
# df_100_cleaned = clean_columns(df_100)

#### Extra analysis for score=100 values

In [38]:
df_100.shape

(48632, 25)

In [39]:
df_100.columns

Index(['Product Id', 'Manufacturer Name', 'Manufacturer PID', 'model',
       'Short Description', 'Long Description', 'Additional description',
       'score', 'matched_pid_left', 'matched_pid_right', 'matched_brand_left',
       'matched_brand_right', 'matched_Product Id', 'matched_model',
       'matched_Manufacturer PID', 'matched_Supplier PID',
       'matched_Manufacturer Name', 'matched_Supplier Name', 'pair',
       'cleaned_Manufacturer Name', 'cleaned_Manufacturer PID',
       'cleaned_model', 'match_Manufacturer Name_cleaned',
       'match_Manufacturer PID_cleaned', 'match_model_cleaned'],
      dtype='object')

In [40]:
df_100['matched_Supplier Name'].value_counts()

matched_Supplier Name
Mahle GmbH                              2898
FESTO PNEUMATIC SA                       543
Festo Corporation                        538
KEYENCE MEXICO S A  DE C V                92
LEINART'S INC                             77
                                        ... 
MEG MACHINERY                              1
UK                                         1
RIMTEC CORP                                1
LYNX MACHINE & TOOL**                      1
325710 Flexschlauch Produktions GmbH       1
Name: count, Length: 189, dtype: int64

In [41]:
df_100_fin=df_100[['pair', 'Manufacturer Name', 'Manufacturer PID', 'model',
       'Short Description', 'matched_pid_left', 'matched_pid_right', 'matched_brand_left',
       'matched_brand_right', 'matched_model', 'matched_Manufacturer PID', 'matched_Manufacturer Name',
       'cleaned_Manufacturer Name', 'cleaned_Manufacturer PID', 'cleaned_model', 'match_Manufacturer Name_cleaned',
       'match_Manufacturer PID_cleaned', 'match_model_cleaned']]

In [42]:
df_100_fin.columns = [col.replace(" ", "_") for col in df_100_fin.columns]

In [43]:
df_100_fin.tail(10)

,pair,Manufacturer_Name,Manufacturer_PID,model,Short_Description,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,matched_model,matched_Manufacturer_PID,matched_Manufacturer_Name,cleaned_Manufacturer_Name,cleaned_Manufacturer_PID,cleaned_model,match_Manufacturer_Name_cleaned,match_Manufacturer_PID_cleaned,match_model_cleaned
925735,"(61080877, 61083589)",ENSHU FFB4010TAD,NaN,ES450,ENSHU ES450 MAGAZIN UJJ JOBBOS,es450,es450,enshuffb4010tad,enshuffb4010tad,ES450,NaN,ENSHU FFB4010TAD,enshuffb4010tad,NaN,es450,enshuffb4010tad,NaN,es450
926042,"(61082486, 61089248)",KEYENCE,NaN,LR-TB2000CL,Allzweck-Lasersensor,lrtb2000cl,lrtb2000cl,keyence,keyence,LR-TB2000CL,NaN,KEYENCE,keyence,NaN,lrtb2000cl,keyence,NaN,lrtb2000cl
926289,"(61065693, 61081738)",SICK,1211477,C4C-EA10510A10000,Sicherheitslichtvorhänge,c4cea10510a10000,c4cea10510a10000,sick,sick,C4C-EA10510A10000,1211477,SICK,sick,1211477,c4cea10510a10000,sick,1211477,c4cea10510a10000
926415,"(61081167, 61081174)",CHIRON,1128703,ZAS200,CHIRON_ZAS200_1128703 O-RING 65X2 MACH.,zas200,zas200,chiron,chiron,ZAS200,1152787,CHIRON,chiron,1128703,zas200,chiron,1152787,zas200
928057,"(61066499, 61066503)",HUSSMANN,BJS 21.58x2.50x0.25,#16,7006098.000 roll no. 16 top 21.58.00x2.5,bjs2158x250x025,bjs2158x250x025,hussmann,hussmann,#3,BJS 21.58x2.50x0.25,HUSSMANN,hussmann,bjs2158x250x025,16,hussmann,bjs2158x250x025,3
928194,"(61066423, 61066424)",HUSSMANN,BJW 28x1.4x0.24,#2,113678 roll no.2 Bottom 28.00x1.40,bjw28x14x024,bjw28x14x024,hussmann,hussmann,#6,BJW 28x1.4x0.24,HUSSMANN,hussmann,bjw28x14x024,2,hussmann,bjw28x14x024,6
928355,"(61066579, 61066599)",HUSSMANN,CB 16.x1.30x0.24,#3,112017 roll no.3 Bottom 16.00x1.30x0.24,cb16x130x024,cb16x130x024,hussmann,hussmann,#1,CB 16.x1.30x0.24,HUSSMANN,hussmann,cb16x130x024,3,hussmann,cb16x130x024,1
929450,"(61025517, 61025526)",MAHLE QUOTATION,5 9120 000,AZ080789,16mm Top Roll Station 12T. AZ080789,az080789,az080789,mahlequotation,mahlequotation,AZ080789,AZ080789(knife-.35mm),MAHLE QUOTATION,mahlequotation,59120000,az080789,mahlequotation,az080789knife35mm,az080789
929451,"(61025509, 61025526)",MAHLE QUOTATION,5 9120 000,AZ080789,16mm Top Roll Station 12T. AZ080789,az080789,az080789,mahlequotation,mahlequotation,AZ080789,5 9010 000,MAHLE QUOTATION,mahlequotation,59120000,az080789,mahlequotation,59010000,az080789
929569,"(61025509, 61025517)",MAHLE QUOTATION,AZ080789(knife-.35mm),AZ080789,BJW 16x2.03x0.22 Cutting Knife SF,az080789,az080789,mahlequotation,mahlequotation,AZ080789,5 9010 000,MAHLE QUOTATION,mahlequotation,az080789knife35mm,az080789,mahlequotation,59010000,az080789


In [44]:
df_100_fin['new_score'] = df_100_fin.apply(calculate_new_score, axis=1)

In [45]:
df_100_fin.tail()

,pair,Manufacturer_Name,Manufacturer_PID,model,Short_Description,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,matched_model,matched_Manufacturer_PID,matched_Manufacturer_Name,cleaned_Manufacturer_Name,cleaned_Manufacturer_PID,cleaned_model,match_Manufacturer_Name_cleaned,match_Manufacturer_PID_cleaned,match_model_cleaned,new_score
928194,"(61066423, 61066424)",HUSSMANN,BJW 28x1.4x0.24,#2,113678 roll no.2 Bottom 28.00x1.40,bjw28x14x024,bjw28x14x024,hussmann,hussmann,#6,BJW 28x1.4x0.24,HUSSMANN,hussmann,bjw28x14x024,2,hussmann,bjw28x14x024,6,67
928355,"(61066579, 61066599)",HUSSMANN,CB 16.x1.30x0.24,#3,112017 roll no.3 Bottom 16.00x1.30x0.24,cb16x130x024,cb16x130x024,hussmann,hussmann,#1,CB 16.x1.30x0.24,HUSSMANN,hussmann,cb16x130x024,3,hussmann,cb16x130x024,1,67
929450,"(61025517, 61025526)",MAHLE QUOTATION,5 9120 000,AZ080789,16mm Top Roll Station 12T. AZ080789,az080789,az080789,mahlequotation,mahlequotation,AZ080789,AZ080789(knife-.35mm),MAHLE QUOTATION,mahlequotation,59120000,az080789,mahlequotation,az080789knife35mm,az080789,67
929451,"(61025509, 61025526)",MAHLE QUOTATION,5 9120 000,AZ080789,16mm Top Roll Station 12T. AZ080789,az080789,az080789,mahlequotation,mahlequotation,AZ080789,5 9010 000,MAHLE QUOTATION,mahlequotation,59120000,az080789,mahlequotation,59010000,az080789,67
929569,"(61025509, 61025517)",MAHLE QUOTATION,AZ080789(knife-.35mm),AZ080789,BJW 16x2.03x0.22 Cutting Knife SF,az080789,az080789,mahlequotation,mahlequotation,AZ080789,5 9010 000,MAHLE QUOTATION,mahlequotation,az080789knife35mm,az080789,mahlequotation,59010000,az080789,67


### 3.3) Analyze for whole dataset

In [46]:
df_cleaned.shape

(240944, 19)

In [47]:
filtered_df_brand = df_cleaned[(df_cleaned['matched_brand_left'] != df_cleaned['matched_brand_right'])]

In [48]:
filtered_df_brand[['matched_brand_left', 'matched_brand_right']]

,matched_brand_left,matched_brand_right
2,carrlane,carrline
684,camlaw,cawlaw
700,norgren,norgreen
779,ppe,NaN
944,solahd,sola
...,...,...
930405,sehi,NaN
930406,sehi,NaN
930407,sehi,NaN
930952,ceadvanced,NaN


In [49]:
filtered_df_pid = df_cleaned[(df_cleaned['matched_pid_left'] != df_cleaned['matched_pid_right'])] 

In [50]:
filtered_df_pid[['matched_pid_left', 'matched_pid_right']]

,matched_pid_left,matched_pid_right
0,cl40bp1,cl70bp1
1,cl40bp1,cl40sbp2
2,cl40bp1,clm10bp1
3,nztpic162150,nztpic162850
4,nztpic162150,nztpic161150
...,...,...
931733,tfm7403,tfm7406
931738,tfm2318,tfm2317
931739,tfm2318,tfm2038
931740,tfm2318,tfm3318


In [51]:
df_scores = clean_columns(df_cleaned)

df_scores = df_scores[['pair', 'score', 'Manufacturer Name', 'Manufacturer PID', 'model',
       'Short Description', 'matched_pid_left', 'matched_pid_right', 'matched_brand_left',
       'matched_brand_right', 'matched_model', 'matched_Manufacturer PID', 'matched_Manufacturer Name',
       'cleaned_Manufacturer Name', 'cleaned_Manufacturer PID', 'cleaned_model', 'match_Manufacturer Name_cleaned',
       'match_Manufacturer PID_cleaned', 'match_model_cleaned']]

df_scores.columns = [col.replace(" ", "_") for col in df_scores.columns]

df_scores['new_score'] = df_scores.apply(calculate_new_score, axis=1)

In [52]:
df_scores.head()

,pair,score,Manufacturer_Name,Manufacturer_PID,model,Short_Description,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,matched_model,matched_Manufacturer_PID,matched_Manufacturer_Name,cleaned_Manufacturer_Name,cleaned_Manufacturer_PID,cleaned_model,match_Manufacturer_Name_cleaned,match_Manufacturer_PID_cleaned,match_model_cleaned,new_score
0,"(61085352, 61085983)",85.71,CARR-LANE,CL-40-BP-1,CL-40-BP-1,"PLUNGER, 1/4 X 20 PN CL-40-BP-1",cl40bp1,cl70bp1,carrlane,carrlane,CL70BP1,CL70BP1,CARR LANE,carrlane,cl40bp1,cl40bp1,carrlane,cl70bp1,cl70bp1,34
1,"(61085952, 61085983)",71.43,CARR-LANE,CL-40-BP-1,CL-40-BP-1,"PLUNGER, 1/4 X 20 PN CL-40-BP-1",cl40bp1,cl40sbp2,carrlane,carrlane,CL-40-SBP-2,CL-40-SBP-2,CARR LANE,carrlane,cl40bp1,cl40bp1,carrlane,cl40sbp2,cl40sbp2,34
2,"(61069715, 61085983)",58.93,CARR-LANE,CL-40-BP-1,CL-40-BP-1,"PLUNGER, 1/4 X 20 PN CL-40-BP-1",cl40bp1,clm10bp1,carrlane,carrline,NaN,CLM-10-BP1,CARR LINE,carrlane,cl40bp1,cl40bp1,carrline,clm10bp1,NaN,34
3,"(61068377, 61072581)",91.67,Yudo,NZTPIC162150,NZTPIC162150,THERMOCOUPLE,nztpic162150,nztpic162850,yudo,yudo,NZTPIC162850,NZTPIC162850,Yudo,yudo,nztpic162150,nztpic162150,yudo,nztpic162850,nztpic162850,34
4,"(61068377, 61068388)",91.67,Yudo,NZTPIC162150,NZTPIC162150,THERMOCOUPLE,nztpic162150,nztpic161150,yudo,yudo,NZTPIC161150,NZTPIC161150,Yudo,yudo,nztpic162150,nztpic162150,yudo,nztpic161150,nztpic161150,34


In [53]:
df_scores.score.describe()

count    240944.000000
mean         84.896631
std          11.108099
min          50.480000
25%          77.780000
50%          85.710000
75%          92.860000
max         100.000000
Name: score, dtype: float64

In [54]:
# Including hightest but not lowest
score_bins = [45, 60, 70, 80, 90, 95, 100]
score_labels = ['46-60', '60-70', '70-80', '80-90', '90-95', '95-100'] 
df_scores['score_interval'] = pd.cut(df_scores['score'], bins=score_bins, labels=score_labels, right=True, include_lowest=False)
grouped = df_scores.groupby('score_interval')['new_score'].describe()
print(grouped)

                  count       mean        std  min   25%    50%    75%    max
score_interval                                                               
46-60            3218.0  47.618397  22.488386  1.0  34.0   34.0   67.0  100.0
60-70           32162.0  40.086562  16.554952  1.0  34.0   34.0   34.0  100.0
70-80           38269.0  50.828974  20.318496  1.0  34.0   34.0   67.0  100.0
80-90           87848.0  49.332483  19.321240  1.0  34.0   34.0   67.0  100.0
90-95           30088.0  54.394676  19.226456  1.0  34.0   67.0   67.0  100.0
95-100          49359.0  86.470796  18.376473  1.0  67.0  100.0  100.0  100.0


In [55]:
df_90_95 = df_scores[(df_scores['score'] > 90) & (df_scores['score'] <= 95)]  # Check values obtained are correct

In [56]:
df_90_95.new_score.describe()

count    30088.000000
mean        54.394676
std         19.226456
min          1.000000
25%         34.000000
50%         67.000000
75%         67.000000
max        100.000000
Name: new_score, dtype: float64

### 3.4) Left/right pid and brand

matched_pid left/right == model/manufacturer pid

matched_brand left/right == manufacturer name

In [57]:
df_scores_copy=df_scores.drop(columns='score_interval')

In [58]:
df_scores.head(1)

,pair,score,Manufacturer_Name,Manufacturer_PID,model,Short_Description,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,...,matched_Manufacturer_PID,matched_Manufacturer_Name,cleaned_Manufacturer_Name,cleaned_Manufacturer_PID,cleaned_model,match_Manufacturer_Name_cleaned,match_Manufacturer_PID_cleaned,match_model_cleaned,new_score,score_interval
0,"(61085352, 61085983)",85.71,CARR-LANE,CL-40-BP-1,CL-40-BP-1,"PLUNGER, 1/4 X 20 PN CL-40-BP-1",cl40bp1,cl70bp1,carrlane,carrlane,...,CL70BP1,CARR LANE,carrlane,cl40bp1,cl40bp1,carrlane,cl70bp1,cl70bp1,34,80-90


In [59]:
df_scores_copy=df_scores.copy()

In [60]:
df_scores_copy.shape

(240944, 21)

In [61]:
df_scores_copy.head(2)

,pair,score,Manufacturer_Name,Manufacturer_PID,model,Short_Description,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,...,matched_Manufacturer_PID,matched_Manufacturer_Name,cleaned_Manufacturer_Name,cleaned_Manufacturer_PID,cleaned_model,match_Manufacturer_Name_cleaned,match_Manufacturer_PID_cleaned,match_model_cleaned,new_score,score_interval
0,"(61085352, 61085983)",85.71,CARR-LANE,CL-40-BP-1,CL-40-BP-1,"PLUNGER, 1/4 X 20 PN CL-40-BP-1",cl40bp1,cl70bp1,carrlane,carrlane,...,CL70BP1,CARR LANE,carrlane,cl40bp1,cl40bp1,carrlane,cl70bp1,cl70bp1,34,80-90
1,"(61085952, 61085983)",71.43,CARR-LANE,CL-40-BP-1,CL-40-BP-1,"PLUNGER, 1/4 X 20 PN CL-40-BP-1",cl40bp1,cl40sbp2,carrlane,carrlane,...,CL-40-SBP-2,CARR LANE,carrlane,cl40bp1,cl40bp1,carrlane,cl40sbp2,cl40sbp2,34,70-80


Compare to see how many times they refer to the model or the Manufacturer PID

In [62]:
# Combines both conditions
df_scores_copy['pid_left'] = (df_scores_copy['matched_pid_left'] == df_scores_copy['cleaned_model']) | \
                            (df_scores_copy['matched_pid_left'] == df_scores_copy['cleaned_Manufacturer_PID'])

df_scores_copy['pid_right'] = (df_scores_copy['matched_pid_right'] == df_scores_copy['match_model_cleaned']) | \
                                       (df_scores_copy['matched_pid_right'] == df_scores_copy['match_Manufacturer_PID_cleaned'])

df_scores_copy['pid_vs_none'] = ~df_scores_copy['pid_left'] & ~df_scores_copy['pid_right']  # When there is no matching between pid and model or manufacturer pid

df_scores_copy['brand_vs_manufacturer_name'] = (df_scores_copy['matched_brand_left'] == df_scores_copy['cleaned_Manufacturer_Name']) | \
                                          (df_scores_copy['matched_brand_right'] == df_scores_copy['match_Manufacturer_Name_cleaned'])

# Summarize counts for each condition
summary = {'PID left matches': df_scores_copy['pid_left'].sum(), 'PID right matches': df_scores_copy['pid_right'].sum(),
    'PID matches None': df_scores_copy['pid_vs_none'].sum(), 'Brand matches Manufacturer Name': df_scores_copy['brand_vs_manufacturer_name'].sum()}

summary_df = pd.DataFrame([summary])
summary_df

,PID left matches,PID right matches,PID matches None,Brand matches Manufacturer Name
0,240299,240259,441,236166


In [63]:
df_scores_copy.shape

(240944, 25)

In [64]:
df_scores_copy.head(2)

,pair,score,Manufacturer_Name,Manufacturer_PID,model,Short_Description,matched_pid_left,matched_pid_right,matched_brand_left,matched_brand_right,...,cleaned_model,match_Manufacturer_Name_cleaned,match_Manufacturer_PID_cleaned,match_model_cleaned,new_score,score_interval,pid_left,pid_right,pid_vs_none,brand_vs_manufacturer_name
0,"(61085352, 61085983)",85.71,CARR-LANE,CL-40-BP-1,CL-40-BP-1,"PLUNGER, 1/4 X 20 PN CL-40-BP-1",cl40bp1,cl70bp1,carrlane,carrlane,...,cl40bp1,carrlane,cl70bp1,cl70bp1,34,80-90,True,True,False,True
1,"(61085952, 61085983)",71.43,CARR-LANE,CL-40-BP-1,CL-40-BP-1,"PLUNGER, 1/4 X 20 PN CL-40-BP-1",cl40bp1,cl40sbp2,carrlane,carrlane,...,cl40bp1,carrlane,cl40sbp2,cl40sbp2,34,70-80,True,True,False,True


In [65]:
# Select important columns for review
important_columns = [
    'score',
    'pid_left',
    'matched_pid_left',
    'cleaned_model',
    'cleaned_Manufacturer_PID',
    'pid_right',
    'matched_pid_right',
    'match_model_cleaned',
    'match_Manufacturer_PID_cleaned',
    'pid_vs_none',
    'brand_vs_manufacturer_name',
    'matched_brand_left',
    'cleaned_Manufacturer_Name',
    'matched_brand_right',
    'match_Manufacturer_Name_cleaned']

df_scores_copy_col=df_scores_copy[important_columns] # Adjust the number of rows as needed

In [66]:
df_scores_copy_col.head()

,score,pid_left,matched_pid_left,cleaned_model,cleaned_Manufacturer_PID,pid_right,matched_pid_right,match_model_cleaned,match_Manufacturer_PID_cleaned,pid_vs_none,brand_vs_manufacturer_name,matched_brand_left,cleaned_Manufacturer_Name,matched_brand_right,match_Manufacturer_Name_cleaned
0,85.71,True,cl40bp1,cl40bp1,cl40bp1,True,cl70bp1,cl70bp1,cl70bp1,False,True,carrlane,carrlane,carrlane,carrlane
1,71.43,True,cl40bp1,cl40bp1,cl40bp1,True,cl40sbp2,cl40sbp2,cl40sbp2,False,True,carrlane,carrlane,carrlane,carrlane
2,58.93,True,cl40bp1,cl40bp1,cl40bp1,True,clm10bp1,NaN,clm10bp1,False,True,carrlane,carrlane,carrline,carrline
3,91.67,True,nztpic162150,nztpic162150,nztpic162150,True,nztpic162850,nztpic162850,nztpic162850,False,True,yudo,yudo,yudo,yudo
4,91.67,True,nztpic162150,nztpic162150,nztpic162150,True,nztpic161150,nztpic161150,nztpic161150,False,True,yudo,yudo,yudo,yudo


In [67]:
df_scores_copy_col[(df_scores_copy_col['pid_vs_none']==True)]

,score,pid_left,matched_pid_left,cleaned_model,cleaned_Manufacturer_PID,pid_right,matched_pid_right,match_model_cleaned,match_Manufacturer_PID_cleaned,pid_vs_none,brand_vs_manufacturer_name,matched_brand_left,cleaned_Manufacturer_Name,matched_brand_right,match_Manufacturer_Name_cleaned
6333,100.00,False,14x112,5591,NaN,False,14x112,5591,NaN,True,True,monoprice,monoprice,monoprice,monoprice
6335,100.00,False,14x112,5591,NaN,False,14x112,5591,NaN,True,True,monoprice,monoprice,monoprice,monoprice
18103,100.00,False,6315skf,NaN,6315,False,6315skf,NaN,6315,True,True,skf,skf,skf,skf
18107,85.71,False,6315skf,NaN,6315,False,6318skf,NaN,6318,True,True,skf,skf,skf,skf
18112,83.33,False,38x116,czc632160,NaN,False,38x112,czc542148,NaN,True,False,grainger,eaglegroup,grainger,eaglegroup
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
866870,85.71,False,1001096,NaN,NaN,False,1001094,NaN,NaN,True,False,rexelgermanygmbhcokg,NaN,rexelgermanygmbhcokg,NaN
866871,85.71,False,1001099,NaN,NaN,False,1001094,NaN,NaN,True,False,rexelgermanygmbhcokg,NaN,rexelgermanygmbhcokg,NaN
866872,85.71,False,1001099,NaN,NaN,False,1001091,NaN,NaN,True,False,rexelgermanygmbhcokg,NaN,rexelgermanygmbhcokg,NaN
893106,92.31,False,pruefstab20mm,prfstab20mm,2022600,False,pruefstab30mm,prfstab30mm,2022602,True,True,sick,sick,sick,sick


In [68]:
df_scores_copy_col[(df_scores_copy_col['brand_vs_manufacturer_name']==False)]

,score,pid_left,matched_pid_left,cleaned_model,cleaned_Manufacturer_PID,pid_right,matched_pid_right,match_model_cleaned,match_Manufacturer_PID_cleaned,pid_vs_none,brand_vs_manufacturer_name,matched_brand_left,cleaned_Manufacturer_Name,matched_brand_right,match_Manufacturer_Name_cleaned
948,100.00,True,6900502,twr20b128,6900502,True,6900502,twr20b128,6900502,False,False,hansturck,turck,hansturck,turck
949,100.00,True,6900502,twr20b128,6900502,True,6900502,twr20b128,6900502,False,False,hansturck,turck,hansturck,turck
962,88.89,True,twr20b128,twr20b128,6900502,True,twr50b128,twr50b128,6900504,False,False,hansturck,turck,hansturck,turck
1058,100.00,True,ni20q14ap6x2v1131,ni20q14ap6x2v1131,4690210,True,ni20q14ap6x2v1131,ni20q14ap6x2v1131,4690210,False,False,hansturck,turck,hansturck,turck
1062,71.43,True,4690210,ni20q14ap6x2v1131,4690210,True,6902810,hs52310,6902810,False,False,hansturck,turck,hansturck,turck
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
928812,76.92,True,afr075003s2p1,NaN,afr075003s2p1,True,af060003s2p1,NaN,af060003s2p1,False,False,apextoolgroup,apex,apextoolgroup,apex
928981,71.43,True,3061673,qs18vp6rbq,3061673,True,3061630,qs18vp6lp,3061630,False,False,hansturck,turck,hansturck,turck
929087,87.50,True,bi4m12evp6xh1141,bi4m12evp6xh1141,4608031,True,bi4m12ap6xh1141,bi4m12ap6xh1141,46070,False,False,hansturck,turck,hansturck,turck
929088,87.50,True,bi4m12evp6xh1141,bi4m12evp6xh1141,4608031,True,bi4m12ap6xh1141,bi4m12ap6xh1141,46070,False,False,hansturck,turck,hansturck,turck


For PID compare matches between model and Manufacturer PID

In [69]:
# Right PID matches
right_model_count = (df_scores_copy['matched_pid_right'] == df_scores_copy['match_model_cleaned']).sum()
right_manufacturer_pid_count = (df_scores_copy['matched_pid_right'] == df_scores_copy['match_Manufacturer_PID_cleaned']).sum()

# Both Right Conditions True
right_both_count = ((df_scores_copy['matched_pid_right'] == df_scores_copy['match_model_cleaned']) &
                    (df_scores_copy['matched_pid_right'] == df_scores_copy['match_Manufacturer_PID_cleaned'])).sum()

# None Right Conditions True
right_none_count = (~(df_scores_copy['matched_pid_right'] == df_scores_copy['match_model_cleaned']) &
                    ~(df_scores_copy['matched_pid_right'] == df_scores_copy['match_Manufacturer_PID_cleaned'])).sum()

# Left PID matches
left_model_count = (df_scores_copy['matched_pid_left'] == df_scores_copy['cleaned_model']).sum()
left_manufacturer_pid_count = (df_scores_copy['matched_pid_left'] == df_scores_copy['cleaned_Manufacturer_PID']).sum()

# Both Left Conditions True
left_both_count = ((df_scores_copy['matched_pid_left'] == df_scores_copy['cleaned_model']) &
                   (df_scores_copy['matched_pid_left'] == df_scores_copy['cleaned_Manufacturer_PID'])).sum()

# None Left Conditions True
left_none_count = (~(df_scores_copy['matched_pid_left'] == df_scores_copy['cleaned_model']) &
                   ~(df_scores_copy['matched_pid_left'] == df_scores_copy['cleaned_Manufacturer_PID'])).sum()

# Print results
print("Right PID Matches:")
print(" - Model Cleaned:", right_model_count)
print(" - Manufacturer PID Cleaned:", right_manufacturer_pid_count)
print(" - Both:", right_both_count)
print(" - None:", right_none_count)

print("\nLeft PID Matches:")
print(" - Model Cleaned:", left_model_count)
print(" - Manufacturer PID Cleaned:", left_manufacturer_pid_count)
print(" - Both:", left_both_count)
print(" - None:", left_none_count)


Right PID Matches:
 - Model Cleaned: 99013
 - Manufacturer PID Cleaned: 154070
 - Both: 12824
 - None: 685

Left PID Matches:
 - Model Cleaned: 99718
 - Manufacturer PID Cleaned: 154515
 - Both: 13934
 - None: 645


### 4) Change the scoring function and compare with pid left/right and brand left/right

In [70]:
df_scores_copy_col.head()

,score,pid_left,matched_pid_left,cleaned_model,cleaned_Manufacturer_PID,pid_right,matched_pid_right,match_model_cleaned,match_Manufacturer_PID_cleaned,pid_vs_none,brand_vs_manufacturer_name,matched_brand_left,cleaned_Manufacturer_Name,matched_brand_right,match_Manufacturer_Name_cleaned
0,85.71,True,cl40bp1,cl40bp1,cl40bp1,True,cl70bp1,cl70bp1,cl70bp1,False,True,carrlane,carrlane,carrlane,carrlane
1,71.43,True,cl40bp1,cl40bp1,cl40bp1,True,cl40sbp2,cl40sbp2,cl40sbp2,False,True,carrlane,carrlane,carrlane,carrlane
2,58.93,True,cl40bp1,cl40bp1,cl40bp1,True,clm10bp1,NaN,clm10bp1,False,True,carrlane,carrlane,carrline,carrline
3,91.67,True,nztpic162150,nztpic162150,nztpic162150,True,nztpic162850,nztpic162850,nztpic162850,False,True,yudo,yudo,yudo,yudo
4,91.67,True,nztpic162150,nztpic162150,nztpic162150,True,nztpic161150,nztpic161150,nztpic161150,False,True,yudo,yudo,yudo,yudo


In [71]:
df_scores_copy_col['new_score'] = df_scores_copy_col.apply(calculate_pid_brand_score, axis=1)

(45, 60]
(60, 70]
(70, 80]  Excluiding the lower limit

In [72]:
score_bins = [45, 60, 70, 80, 90, 95, 100]
score_labels = ['46-60', '60-70', '70-80', '80-90', '90-95', '95-100'] 
df_scores_copy_col['score_interval'] = pd.cut(df_scores_copy_col['score'], bins=score_bins, labels=score_labels, right=True, include_lowest=False)
grouped = df_scores_copy_col.groupby('score_interval')['new_score'].describe()
print(grouped)

                  count       mean        std   min    25%    50%    75%  \
score_interval                                                             
46-60            3218.0  39.418894  20.426090   0.0   50.0   50.0   50.0   
60-70           32162.0  48.397177   8.807640   0.0   50.0   50.0   50.0   
70-80           38269.0  48.954768   7.153352   0.0   50.0   50.0   50.0   
80-90           87848.0  49.813883   3.044883   0.0   50.0   50.0   50.0   
90-95           30088.0  49.951808   1.551566   0.0   50.0   50.0   50.0   
95-100          49359.0  99.142000   6.493433  50.0  100.0  100.0  100.0   

                  max  
score_interval         
46-60            50.0  
60-70            50.0  
70-80            50.0  
80-90            50.0  
90-95            50.0  
95-100          100.0  


In [73]:
df_scores_copy_col.head(10)

,score,pid_left,matched_pid_left,cleaned_model,cleaned_Manufacturer_PID,pid_right,matched_pid_right,match_model_cleaned,match_Manufacturer_PID_cleaned,pid_vs_none,brand_vs_manufacturer_name,matched_brand_left,cleaned_Manufacturer_Name,matched_brand_right,match_Manufacturer_Name_cleaned,new_score,score_interval
0,85.71,True,cl40bp1,cl40bp1,cl40bp1,True,cl70bp1,cl70bp1,cl70bp1,False,True,carrlane,carrlane,carrlane,carrlane,50,80-90
1,71.43,True,cl40bp1,cl40bp1,cl40bp1,True,cl40sbp2,cl40sbp2,cl40sbp2,False,True,carrlane,carrlane,carrlane,carrlane,50,70-80
2,58.93,True,cl40bp1,cl40bp1,cl40bp1,True,clm10bp1,NaN,clm10bp1,False,True,carrlane,carrlane,carrline,carrline,0,46-60
3,91.67,True,nztpic162150,nztpic162150,nztpic162150,True,nztpic162850,nztpic162850,nztpic162850,False,True,yudo,yudo,yudo,yudo,50,90-95
4,91.67,True,nztpic162150,nztpic162150,nztpic162150,True,nztpic161150,nztpic161150,nztpic161150,False,True,yudo,yudo,yudo,yudo,50,90-95
5,83.33,True,nztpic162150,nztpic162150,nztpic162150,True,nztpic161750,NaN,nztpic161750,False,True,yudo,yudo,yudo,yudo,50,80-90
6,75.00,True,q23sp6rq,q23sp6rq,q23sp6rq,True,q23sp6d,q23sp6d,q23sp6d,False,True,banner,banner,banner,banner,50,70-80
7,100.00,True,r165111320,NaN,r165111320,True,r165111320,NaN,r165111320,False,True,bosch,bosch,bosch,bosch,100,95-100
8,100.00,True,r165111320,NaN,r165111320,True,r165111320,NaN,r165111320,False,True,bosch,bosch,bosch,boschrexroth,100,95-100
13,90.00,True,r165111320,NaN,r165111320,True,165111320,NaN,165111320,False,True,bosch,bosch,bosch,boschrexroth,50,80-90


In [74]:
df_scores_copy_col.new_score.describe()

count    240944.000000
mean         59.471910
std          21.056975
min           0.000000
25%          50.000000
50%          50.000000
75%          50.000000
max         100.000000
Name: new_score, dtype: float64

In [75]:
df_scores_copy_col[df_scores_copy_col['new_score']==0]

,score,pid_left,matched_pid_left,cleaned_model,cleaned_Manufacturer_PID,pid_right,matched_pid_right,match_model_cleaned,match_Manufacturer_PID_cleaned,pid_vs_none,brand_vs_manufacturer_name,matched_brand_left,cleaned_Manufacturer_Name,matched_brand_right,match_Manufacturer_Name_cleaned,new_score,score_interval
2,58.93,True,cl40bp1,cl40bp1,cl40bp1,True,clm10bp1,NaN,clm10bp1,False,True,carrlane,carrlane,carrline,carrline,0,46-60
700,65.71,True,pra8040m50,pra8040m50,pra8040m50,True,pra8080m90,pra8080m90,pra8080m90,False,True,norgren,norgren,norgreen,norgreen,0,60-70
779,60.00,True,bp116,bp116,bp116,True,b1165,b1165,NaN,False,True,ppe,ppe,NaN,,0,46-60
944,57.58,True,sdn4024480c,NaN,sdn4024480c,True,sdn4024480,sdn4024480,sdn4024480,False,True,solahd,solahd,sola,sola,0,46-60
1219,65.71,True,dm92016m35,dm92016m35,dm92016m35,True,dm92032m35,dm92032m35,dm92032m35,False,True,norgren,norgren,norgreen,norgreen,0,60-70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930405,66.67,True,700104,NaN,700104,True,70014064,NaN,70014064,False,True,sehi,sehi,NaN,,0,60-70
930406,66.67,True,700104,NaN,700104,True,75001043,NaN,75001043,False,True,sehi,sehi,NaN,,0,60-70
930407,66.67,True,700104,NaN,700104,True,70011074,NaN,70011074,False,True,sehi,sehi,NaN,,0,60-70
930952,62.50,True,78519201,ni9375,78519201,True,7825190,NaN,7825190,False,True,ceadvanced,ceadvanced,NaN,,0,60-70


### 5) Analyze 90-95 interval

In [76]:
df_95_100 = df_scores_copy_col[df_scores_copy_col['score_interval'] == '90-95']

In [77]:
df_95_100

,score,pid_left,matched_pid_left,cleaned_model,cleaned_Manufacturer_PID,pid_right,matched_pid_right,match_model_cleaned,match_Manufacturer_PID_cleaned,pid_vs_none,brand_vs_manufacturer_name,matched_brand_left,cleaned_Manufacturer_Name,matched_brand_right,match_Manufacturer_Name_cleaned,new_score,score_interval
3,91.67,True,nztpic162150,nztpic162150,nztpic162150,True,nztpic162850,nztpic162850,nztpic162850,False,True,yudo,yudo,yudo,yudo,50,90-95
4,91.67,True,nztpic162150,nztpic162150,nztpic162150,True,nztpic161150,nztpic161150,nztpic161150,False,True,yudo,yudo,yudo,yudo,50,90-95
229,91.67,True,cyvpty104500,cyvpty104500,cyvpty104500,True,cyvpty103500,cyvpty103500,cyvpty103500,False,True,yudo,yudo,yudo,yudo,50,90-95
243,90.91,True,vte184p8240,vte184p8240,6013119,True,vte184p4240,vte184p4240,6013263,False,True,sick,sick,sick,sick,50,90-95
577,93.75,True,554bb400mt000061,554bb400mt000061,554bb400mt00006124,True,554ba400mt000061,554ba400mt000061,554ba400mt00006124,False,True,emerson,emerson,emerson,emerson,50,90-95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931600,90.91,True,as1301fm506,NaN,as1301fm506,True,as1201fm506,NaN,as1201fm506,False,True,smc,smc,smc,smc,50,90-95
931601,90.91,True,as1301fm506,NaN,as1301fm506,True,as1201fm506,NaN,as1201fm506,False,True,smc,smc,smc,smc,50,90-95
931602,90.91,True,as1301fm506,NaN,as1301fm506,True,as1201fm506,NaN,as1201fm506,False,True,smc,smc,smc,smc,50,90-95
931603,90.91,True,as1301fm506,NaN,as1301fm506,True,as1201fm506,NaN,as1201fm506,False,True,smc,smc,smc,smc,50,90-95


In [78]:
df_95_100.matched_brand_left.value_counts()

matched_brand_left
festo               8572
siemens             8144
balluff             1717
smc                 1167
fanuc               1162
                    ... 
tunkers                1
controltechkft         1
airtificialweber       1
nord                   1
komet                  1
Name: count, Length: 597, dtype: int64

##### 5.1) Festo

In [79]:
df_festo=df_95_100[df_95_100['matched_brand_left']=='festo']

In [80]:
df_festo

,score,pid_left,matched_pid_left,cleaned_model,cleaned_Manufacturer_PID,pid_right,matched_pid_right,match_model_cleaned,match_Manufacturer_PID_cleaned,pid_vs_none,brand_vs_manufacturer_name,matched_brand_left,cleaned_Manufacturer_Name,matched_brand_right,match_Manufacturer_Name_cleaned,new_score,score_interval
1809,90.91,True,dgst630y12a,dgst630y12a,8085159,True,dgst1630y12a,dgst1630y12a,8085176,False,True,festo,festo,festo,festo,50,90-95
1810,90.91,True,dgst630y12a,dgst630y12a,8085159,True,dgst1630y12a,dgst1630y12a,8085176,False,True,festo,festo,festo,festo,50,90-95
1811,90.91,True,dgst630y12a,dgst630y12a,8085159,True,dgst1630y12a,dgst1630y12a,8085176,False,True,festo,festo,festo,festo,50,90-95
1818,90.91,True,dgst630y12a,dgst630y12a,8085159,True,dgst830y12a,dgst830y12a,8085162,False,True,festo,festo,festo,festo,50,90-95
6049,92.31,True,dsbc4060cppva,dsbc4060cppva,NaN,True,dsbc5060cppva,dsbc5060cppva,1463770,False,True,festo,festo,festo,festo,50,90-95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929888,90.91,True,adngf1240pa,adngf1240pa,1240,True,adngf1210pa,adngf1210pa,554206,False,True,festo,festo,festo,festo,50,90-95
929889,90.91,True,adngf1240pa,adngf1240pa,1240,True,adngf1220pa,adngf1220pa,554208,False,True,festo,festo,festo,festo,50,90-95
929890,90.91,True,adngf1240pa,adngf1240pa,1240,True,adngf1220pa,adngf1220pa,554208,False,True,festo,festo,festo,festo,50,90-95
929891,90.91,True,adngf1240pa,adngf1240pa,1240,True,adngf1220pa,adngf1220pa,554208,False,True,festo,festo,festo,festo,50,90-95


The model numbers DGST-6-30-Y12A and DGST-16-30-Y12A refer to two distinct mini slide actuators produced by Festo.

Key Differences:

Piston Diameter:

DGST-6-30-Y12A: 6 mm
DGST-16-30-Y12A: 16 mm
Theoretical Force at 6 bar (Advancing):

DGST-6-30-Y12A: 34 N
DGST-16-30-Y12A: 241 N
Product Weight:

DGST-6-30-Y12A: 131.8 g
DGST-16-30-Y12A: 576 g

##### 5.2) Siemens

In [168]:
df_siemnens=df_95_100[df_95_100['matched_brand_left']=='siemens']

In [169]:
df_siemnens

,score,pid_left,matched_pid_left,cleaned_model,cleaned_Manufacturer_PID,pid_right,matched_pid_right,match_model_cleaned,match_Manufacturer_PID_cleaned,pid_vs_none,brand_vs_manufacturer_name,matched_brand_left,cleaned_Manufacturer_Name,matched_brand_right,match_Manufacturer_Name_cleaned,new_score,score_interval
5322,91.67,True,3rh29111ha11,3rh29111ha11,3rh29111ha11,True,3rh29111ha13,NaN,3rh29111ha13,False,True,siemens,siemens,siemens,siemens,50,90-95
5323,91.67,True,3rh29111ha11,3rh29111ha11,3rh29111ha11,True,3rh29111ha13,NaN,3rh29111ha13,False,True,siemens,siemens,siemens,siemens,50,90-95
5324,91.67,True,3rh29111ha11,3rh29111ha11,3rh29111ha11,True,3rh29111ha13,NaN,3rh29111ha13,False,True,siemens,siemens,siemens,siemens,50,90-95
5833,93.75,True,6es71517aa210abo,6es71517aa210abo,NaN,True,6es71517aa210ab0,NaN,6es71517aa210ab0,False,True,siemens,siemens,siemens,siemens,50,90-95
6202,93.75,True,6es71556au010bn0,NaN,6es71556au010bn0,True,6es71556aa010bn0,NaN,6es71556aa010bn0,False,True,siemens,siemens,siemens,siemens,50,90-95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929866,93.75,True,1fk70225ak711va3,NaN,1fk70225ak711va3,True,1fk70225ak711la3,NaN,1fk70225ak711la3,False,True,siemens,siemens,siemens,siemens,50,90-95
929867,93.75,True,1fk70225ak711va3,NaN,1fk70225ak711va3,True,1fk70225ak711la3,NaN,1fk70225ak711la3,False,True,siemens,siemens,siemens,siemens,50,90-95
930078,93.75,True,6fx80025ca011bb0,6fx80025ca011bb0,NaN,True,6fx80025ca011bj0,NaN,6fx80025ca011bj0,False,True,siemens,siemens,siemens,siemens,50,90-95
930079,93.75,True,6fx80025ca011bb0,6fx80025ca011bb0,NaN,True,6fx80025ca011ba0,NaN,6fx80025ca011ba0,False,True,siemens,siemens,siemens,siemens,50,90-95


The Siemens 3RH2911-1HA11 is an auxiliary switch block designed for front mounting on SIRIUS 3RT2 contactors and 3RH2 contactor relays. It features 1 normally open (NO) and 1 normally closed (NC) contact, with screw terminals for secure connections. 
SIEMENS INDUSTRY SUPPORT

Regarding the 3RH2911-1HA13, I couldn't locate specific information in the available sources. However, based on Siemens' typical product numbering conventions, the difference between these two models likely pertains to the configuration of the auxiliary contacts. The last two digits in Siemens' part numbers often indicate the contact arrangement. For instance, '11' denotes 1 NO + 1 NC. Therefore, '13' might represent a different combination, such as 1 NO + 3 NC or 1 NO + 3 NO.

Use OpenAI API key to analyze the differences between 2 products and also to be able to make a decision on whether they are the same or not

In [88]:
query = "Are 3rh29111ha11 and 3rh29111ha13 from siemens the same product??"
result = siemens.process_query(query)
print(result)

3RH2911-1HA11 and 3RH2911-1HA13 are auxiliary switch blocks produced by Siemens. They are not exactly the same as they have slight variations in their design and features. Here are some of their specifications:

1. Siemens Sirius Auxiliary Switch Block 3RH2911-1HA11:
   - It has 2 NO (Normally Open) auxiliary contacts.
   - It comes in screw type terminal connections.
   - It can be mounted side by side.
   - It is executed as single-hole type plug-in circuit.
   - Its actuating voltage is 240 volts.
   - It has an IP20 degree of protection.
   - Its application includes use with 3RT101...3RT104, 3TB44...3TB48 Contactors and 3RV1 Circuit Breakers.
   
2. Siemens Sirius Auxiliary Switch Block 3RH2911-1HA13:
   - It has 1 NO (Normally Open) and 1 NC (Normally Closed) auxiliary contacts.
   - It also comes in screw type terminal connections.
   - It can also be mounted side by side.
   - It is executed as single-hole type plug-in circuit.
   - Its actuating voltage is 240 volts.
   - It h

In [91]:
product1 = "Siemens 3RH29111HA11"
product2 = "Siemens 3RH29111HA13"
comparison_result = siemens.compare_products(product1, product2)
print(f"Are the products the same? {comparison_result}")

Are the products the same? Yes


### 6) Analyze lower scores interval

In [93]:
df_60_70 = df_scores_copy_col[df_scores_copy_col['score_interval'] == '60-70']

In [95]:
df_60_70[df_60_70['new_score']==0]

,score,pid_left,matched_pid_left,cleaned_model,cleaned_Manufacturer_PID,pid_right,matched_pid_right,match_model_cleaned,match_Manufacturer_PID_cleaned,pid_vs_none,brand_vs_manufacturer_name,matched_brand_left,cleaned_Manufacturer_Name,matched_brand_right,match_Manufacturer_Name_cleaned,new_score,score_interval
700,65.71,True,pra8040m50,pra8040m50,pra8040m50,True,pra8080m90,pra8080m90,pra8080m90,False,True,norgren,norgren,norgreen,norgreen,0,60-70
1219,65.71,True,dm92016m35,dm92016m35,dm92016m35,True,dm92032m35,dm92032m35,dm92032m35,False,True,norgren,norgren,norgreen,norgreen,0,60-70
1393,61.67,True,7006526000,NaN,7006526000,True,7006278000,NaN,7006278000,False,True,buildtoprint,buildtoprint,builttoprint,builttoprint,0,60-70
1416,63.10,True,draw4009952001,NaN,draw4009952001,True,draw4009968me001,draw4009968me001,draw4009968me001,False,True,buildtoprint,buildtoprint,builttoprint,builttoprint,0,60-70
1640,68.89,True,311ms1450,311ms1450,311ms1450,True,311ms1850,311ms1850,NaN,False,True,banda,banda,bando,bando,0,60-70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
928006,66.67,True,it001188,it001188,it001188det4dept042sht4,True,it001858,it001858,it001858det2dept045sht10,False,True,buildtoprint,buildtoprint,builttoprint,builttoprint,0,60-70
930405,66.67,True,700104,NaN,700104,True,70014064,NaN,70014064,False,True,sehi,sehi,NaN,,0,60-70
930406,66.67,True,700104,NaN,700104,True,75001043,NaN,75001043,False,True,sehi,sehi,NaN,,0,60-70
930407,66.67,True,700104,NaN,700104,True,70011074,NaN,70011074,False,True,sehi,sehi,NaN,,0,60-70
